In [1]:
import numpy as np
import pandas as pd

In [2]:
seq = pd.read_csv("seq.csv")

# Viterbi Algorithm 

In [3]:
o = np.array(["a", "b", "c"]) # observation space
s = np.array(["St1", "St2"]) # state space
y = np.asarray(seq.iloc[:, 2].copy()) # observation sequence
x_true = np.asarray(seq.iloc[:, 1].copy()) # true result
a = np.array([[0.981, 0.007],[0.019, 0.992]]) # transition probability matrix
b = np.array([[0.076, 0.802, 0.122], [0.179, 0.151, 0.669]]) # emission probability matrix
p = np.array([0.723, 0.277]) # prior probability

In [4]:
y[y == "a"] = 0
y[y == "b"] = 1
y[y == "c"] = 2
x_true[x_true == "St1"] = 0
x_true[x_true == "St2"] = 1

In [5]:
def viterbi(o, s, y, a, b, p):
    
    k = a.shape[0]    
    t = len(y)
    tState = np.empty((k, t), 'd')
    tIndex = np.empty((k, t), 'B')
    x = np.empty(t, 'B')
    
    tState[:, 0] = p * b[:, y[0]]
    tIndex[:, 0] = 0

    for i in range(1, t):
        tState[:, i] = np.max(tState[:, i-1] * a.T * b[np.newaxis, :, y[i]].T, 1)
        tIndex[:, i] = np.argmax(tState[:, i-1] * a.T * b[np.newaxis, :, y[i]].T, 1)

    x[-1] = np.argmax(tState[:, t-1])
    for i in reversed(range(1, t)):
        x[i - 1] = tIndex[x[i], i]

    return x, tState, tIndex

In [6]:
x, tState, tIndex = viterbi(o, s, y, a, b, p)

In [7]:
a = x == 0
b = x_true == 0
Tp = sum(a == b)

a = x == 1
b = x_true == 1
Tn = sum(a == b)

a = x == 0
b = x_true == 1
Fp = sum(a == b)

a = x == 1
b = x_true == 0
Fn = sum(a == b)

In [8]:
Tn / (Tn+Fp)

0.9925

In [9]:
Tp / (Tp+Fn)

0.9925

# Forward–backward algorithm

In [10]:
o = np.array(["a", "b", "c"]) # observation space
s = np.array(["St1", "St2"]) # state space
y = np.asarray(seq.iloc[:, 2].copy()) # observation sequence
e = 2 # number of end state
x_true = np.asarray(seq.iloc[:, 1].copy()) # true result
a = np.array([[0.981, 0.007, 0.01], [0.019, 0.992, 0.01]]) # transition probability matrix
b = np.array([[0.076, 0.802, 0.122], [0.179, 0.151, 0.669]]) # emission probability matrix
p = np.array([0.723, 0.277]) # prior probability

In [11]:
def forward_backward(o, s, p, a, b, e):
    """
    forward_backward(o, s, p, a, b, e)
    
    Arguments:
    
    o - observation space
    s - state space 
    p - prior probability
    a - transition probability matrix
    b - emission probability matrix
    e - number of end state
    
    Description:
    
    The forward–backward algorithm is an inference algorithm for hidden Markov models which computes the posterior marginals of all hidden state variables given a sequence of observations/emissions
    """
    # arrays
    
    forward = []
    forward_previous = []
    backward = []
    backward_previous = []
    posterior = []

    
    # computing forward probabilities

    for i in range(len(o)):
        forward_current = []
        for st in range(len(s)):
            if i == 0:
                previous_forward_sum = p[st]
            else:
                previous_forward_sum = sum(forward_previous[k]*a[k, st] for k in range(len(s)))            
            forward_current.append(b[st, i] * previous_forward_sum)
        forward.append(forward_current)
        forward_previous = forward_current
    p_forward = sum(forward_current[k] * a[k, e] for k in range(len(s)))
        
    print(forward)
    print(forward_previous)
    print(p_forward)
    
    # computing backward probabilities
    
    for i in range(len(o)):
        backward_current = []
        for st in range(len(s)):
            if i == 0:
                backward_current.append(a[st, e])
            else:
                backward_current.append(sum(a[st, l] * b[l, 0-i] * backward_previous[l] for l in range(len(s))))
        backward.insert(0, backward_current)
        backward_previous = backward_current
    p_backward = sum(p[l] * b[l, 0] * backward_current[l] for l in range(len(s)))
    
    
    # computing smoothed values
    
    assert round(p_forward, 5) == round(p_backward, 5)
    for i in range(len(o)):
        posterior.append([forward[i][st] * backward[i][st] / p_forward for st in range(len(s))])
    
    
    # exit function
    
    return posterior

In [12]:
forward_backward(o, s, p, a, b, e)

[[0.054948, 0.049583], [0.04398654413, 0.007485216772000001], [0.005281748307044155, 0.005173538126465047]]
[0.005281748307044155, 0.005173538126465047]
0.00010455286433509202


[[0.5179296116585203, 0.4820703883414798],
 [0.5232172830000005, 0.4767827169999996],
 [0.5051749027282645, 0.49482509727173546]]

In [13]:
states = ('St1', 'St2')
end_state = 'E'
 
observations = ('a', 'b', 'b')
 
start_probability = {'St1': 0.723, 'St2': 0.277}
 
transition_probability = {
   'St1' : {'St1': 0.981, 'St2': 0.007, 'E': 0.01},
   'St2' : {'St1': 0.019, 'St2': 0.992, 'E': 0.01},
   }
 
emission_probability = {
   'St1' : {'a': 0.076, 'b': 0.802, 'c': 0.122},
   'St2' : {'a': 0.179, 'b': 0.151, 'c': 0.669},
   }

def fwd_bkw(observations, states, start_prob, trans_prob, emm_prob, end_st):
    # forward part of the algorithm
    fwd = []
    f_prev = {}
    for i, observation_i in enumerate(observations):
        f_curr = {}
        for st in states:
            if i == 0:
                # base case for the forward part
                prev_f_sum = start_prob[st]
            else:
                prev_f_sum = sum(f_prev[k]*trans_prob[k][st] for k in states)

            f_curr[st] = emm_prob[st][observation_i] * prev_f_sum

        fwd.append(f_curr)
        f_prev = f_curr

    p_fwd = sum(f_curr[k] * trans_prob[k][end_st] for k in states)

    
    print(fwd)
    print(f_prev)
    print(p_fwd)
    
    # backward part of the algorithm
    bkw = []
    b_prev = {}
    for i, observation_i_plus in enumerate(reversed(observations[1:]+(None,))):
        b_curr = {}
        for st in states:
            if i == 0:
                # base case for backward part
                b_curr[st] = trans_prob[st][end_st]
            else:
                b_curr[st] = sum(trans_prob[st][l] * emm_prob[l][observation_i_plus] * b_prev[l] for l in states)

        bkw.insert(0,b_curr)
        b_prev = b_curr
    
    p_bkw = sum(start_prob[l] * emm_prob[l][observations[0]] * b_curr[l] for l in states)

    # merging the two parts
    posterior = []
    for i in range(len(observations)):
        posterior.append({st: fwd[i][st] * bkw[i][st] / p_fwd for st in states})
    
    assert round(p_fwd, 5) == round(p_bkw, 5)
    return fwd, bkw, posterior

fwd_bkw(observations, states, start_probability, transition_probability, emission_probability, end_state)

[{'St1': 0.054948, 'St2': 0.049583}, {'St1': 0.04398654413, 'St2': 0.007485216772000001}, {'St1': 0.034721001165978795, 'St2': 0.0011677193678568342}]
{'St1': 0.034721001165978795, 'St2': 0.0011677193678568342}
0.0003588872053383563


([{'St1': 0.054948, 'St2': 0.049583},
  {'St1': 0.04398654413, 'St2': 0.007485216772000001},
  {'St1': 0.034721001165978795, 'St2': 0.0011677193678568342}],
 [{'St1': 0.006200004887880001, 'St2': 0.00036724959681999994},
  {'St1': 0.00787819, 'St2': 0.0016502999999999997},
  {'St1': 0.01, 'St2': 0.01}],
 [{'St1': 0.9492616719452052, 'St2': 0.0507383280547949},
  {'St1': 0.9655801236291346, 'St2': 0.03441987637086537},
  {'St1': 0.9674627751982433, 'St2': 0.032537224801756776}])

In [ ]:
[[0.054948, 0.049583], [0.04398654413, 0.007485216772000001], [0.005281748307044155, 0.005173538126465047]]
[0.005281748307044155, 0.005173538126465047]
0.00010455286433509202